In [ ]:
import numpy as np
import random
from skimage.measure import regionprops
import matplotlib.pyplot as plt
from PIL import Image

# GA Parameters
POPULATION_SIZE = 20
NUM_GENERATIONS = 100
MUTATION_RATE = 0.1

def generate_population(image_shape, population_size):
    population = []
    for _ in range(population_size):
        p = random.randint(1, image_shape[0])  # Random block size in the range [1, image_height]
        q = random.randint(1, image_shape[1])  # Random block size in the range [1, image_width]
        population.append((p, q))
    return population


def evolve(population, image):
    next_generation = []
    while len(next_generation) < len(population):
        # Selection: Select two parents from the current population
        parent1, parent2 = random.choices(population, k=2)

        # Crossover: Perform crossover to create two offspring
        offspring1, offspring2 = crossover(parent1, parent2)

        # Mutation: Apply mutation to the offspring
        offspring1 = mutate(offspring1, image.shape)
        offspring2 = mutate(offspring2, image.shape)

        # Add offspring to the next generation
        next_generation.extend([offspring1, offspring2])

    return next_generation

def crossover(parent1, parent2):
    p1_p, p1_q = parent1
    p2_p, p2_q = parent2

    # Perform one-point crossover
    crossover_point = random.randint(1, 2)
    if crossover_point == 1:
        offspring1 = (p1_p, p2_q)
        offspring2 = (p2_p, p1_q)
    else:
        offspring1 = (p2_p, p1_q)
        offspring2 = (p1_p, p2_q)

    return offspring1, offspring2


def mutate(individual, image_shape):
    p, q = individual

    # Randomly mutate one dimension of the block size
    mutation_dimension = random.randint(0, 1)
    if mutation_dimension == 0:
        p = random.randint(1, image_shape[0])
    else:
        q = random.randint(1, image_shape[1])

    mutated_individual = (p, q)
    return mutated_individual



def calculate_compression_ratio(image, block_size):
    # Create overlapping blocks from the image
    p, q = block_size
    shape = (image.shape[0] - p + 1, image.shape[1] - q + 1, p, q)
    strides = image.strides + image.strides
    blocks = np.lib.stride_tricks.as_strided(image, shape=shape, strides=strides)

    # Calculate the total number of pixels in the original image
    original_pixels = np.prod(image.shape)

    # Calculate the total number of pixels in the segmented image
    segmented_pixels = np.prod(blocks.shape[:2]) * np.prod(block_size)

    # Calculate the compression ratio
    compression_ratio = original_pixels / segmented_pixels

    return compression_ratio, blocks

def visualize_segmentation(image, blocks):
    plt.figure(figsize=(8, 4))

    plt.subplot(1, 2, 1)
    plt.imshow(image, cmap='gray')
    plt.title('Original Image')

    plt.subplot(1, 2, 2)
    plt.imshow(blocks.mean(axis=(-1, -2)), cmap='gray')
    plt.title('Segmented Image')

    plt.tight_layout()
    plt.show()

def find_optimum_block_size(image):
    # Initialize population
    population = generate_population(image.shape, POPULATION_SIZE)

    for _ in range(NUM_GENERATIONS):
        # Calculate fitness for each individual
        fitness_values = [fitness(individual, image) for individual in population]

        # Sort population by fitness in descending order
        population = [x for _, x in sorted(zip(fitness_values, population), reverse=True)]

        # Select the fittest individual
        fittest_individual = population[0]

        # Generate the next generation
        population = evolve(population, image)

    compression_ratio, blocks = calculate_compression_ratio(image, fittest_individual)
    return fittest_individual, compression_ratio, blocks

def fitness(individual, image):
    compression_ratio, _ = calculate_compression_ratio(image, individual)
    return compression_ratio

# Example usage
# Load an image from your PC
image_path = '/content/gdrive/My Drive/image.jpeg'
image = np.array(Image.open(image_path).convert('L'))

# Convert the image to binary (optional)
binary_image = image > 127

# Find the optimum block size using a Genetic Algorithm
optimum_block_size, compression_ratio, segmented_image = find_optimum_block_size(binary_image)
print("Optimum Block Size:", optimum_block_size)
print("Compression Ratio:", compression_ratio)

# Calculate the relative data redundancy
relative_data_redundancy = 1 - (1 / compression_ratio)
print("Relative Data Redundancy:", relative_data_redundancy)

Optimum Block Size: (187, 1591)
Compression Ratio: 0.0006364312372941227
Relative Data Redundancy: -1570.26165625
